# AutoInland Vehicle Insurance Claim Challenge StarterNotebook - Python

This is a simple starter notebook to get started with the AutoInland Vehicle Insurance Claim Challenge on Zindi.

This notebook covers:
- Loading the data
- Simple EDA and an example of feature enginnering
- Data preprocessing and data wrangling
- Creating a simple model
- Making a submission
- Some tips for improving your score

### Importing libraries

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from tqdm.notebook import tqdm_notebook
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import warnings
warnings.filterwarnings('ignore')

### Read files

In [2]:
# Load files into a pandas dataframe
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
ss = pd.read_csv('SampleSubmission.csv')

## Eploratory Data Analysis

In [ ]:
# Preview the first five rows of the train set
train.head()

In [ ]:
# Preview the first five rows of the test set
test.head(20)
# test["Car_Category"].value_counts()

In [ ]:
# Preview the first five rows of the sample submission file
ss.head()

In [ ]:
# Check the shape of the train and test sets
print(f'The shape of the train set is: {train.shape}\nThe shape of the test set is: {test.shape}')

In [ ]:
# Check if there any missing values in train set
ax = train.isna().sum().sort_values().plot(kind = 'barh', figsize = (10, 7))
plt.title('Percentage of Missing Values Per Column in Train Set', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/train.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

In [ ]:
train["Gender"].value_counts()

In [ ]:
train["State"].value_counts()

In [ ]:
train["LGA_Name"].value_counts()

In [ ]:
train["Subject_Car_Colour"].value_counts()

In [ ]:
train["ID"].value_counts()

In [ ]:
train.isnull().sum()

In [ ]:
train.columns

In [ ]:
test.columns

### Imputing Gender with Mode

In [3]:
train['Gender'] = train['Gender'].fillna(train['Gender'].mode()[0])
test['Gender'] = test['Gender'].fillna(test['Gender'].mode()[0])

### Imputing Age

In [4]:
def clean_age(age):
    if pd.isna(age) or age < 0 or age > 100:
        return np.nan
    return age

train['Age'] = train['Age'].apply(clean_age)
test['Age'] = test['Age'].apply(clean_age)


age_average = train['Age'].mean()
age_average_test = test['Age'].mean()

train['Age'].fillna(age_average, inplace=True)
test['Age'].fillna(age_average_test, inplace=True)

### Replacing Dots

In [5]:
train['Subject_Car_Make'] = train['Subject_Car_Make'].replace('.', np.nan)

### Backfill Method

In [6]:
def backfill_data(data, columns):
    for column in columns:
        data[column] = data[column].fillna(method='bfill')
    return data

                   
backfill_columns = ['Car_Category', 'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State', 'Gender']
    
backfilled_data = backfill_data(train, backfill_columns)

print("Original Data:")
print(train)
print("\nBackfilled Data:")
print(backfilled_data)


Original Data:
               ID Policy Start Date Policy End Date  Gender        Age  \
0      ID_0040R73        2010-05-14      2011-05-13    Male  30.000000   
1      ID_0046BNK        2010-11-29      2011-11-28  Female  79.000000   
2      ID_005QMC3        2010-03-21      2011-03-20    Male  43.000000   
3      ID_0079OHW        2010-08-21      2011-08-20    Male   2.000000   
4      ID_00BRP63        2010-08-29      2010-12-31  Entity  20.000000   
...           ...               ...             ...     ...        ...   
12074  ID_ZZA1SES        2010-05-25      2011-05-24  Female  30.000000   
12075  ID_ZZDAC3K        2010-10-03      2011-10-02  Female  59.000000   
12076  ID_ZZIU2XC        2010-10-10      2011-10-08    Male  34.000000   
12077  ID_ZZRQ1NF        2010-02-27      2011-02-26    Male  41.153991   
12078  ID_ZZWRIIE        2010-07-01      2011-06-30  Female  46.000000   

      First Transaction Date  No_Pol Car_Category Subject_Car_Colour  \
0                 2010-0

In [7]:
def backfill_data(data, columns):
    for column in columns:
        data[column] = data[column].fillna(method='bfill')
    return data

                   
backfill_columns = ['Car_Category', 'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State', 'Gender']
    
backfilled_data = backfill_data(test, backfill_columns)

print("Original Data:")
print(test)
print("\nBackfilled Data:")
print(backfilled_data)

Original Data:
              ID Policy Start Date Policy End Date  Gender        Age  \
0     ID_01QM0NU        2010-10-23      2011-10-22  Female  46.000000   
1     ID_024NJLZ        2010-10-14      2011-10-13    Male  32.000000   
2     ID_02NOVWQ        2010-08-29      2011-08-28  Female  45.000000   
3     ID_02VSP68        2010-06-13      2011-06-12  Female  58.000000   
4     ID_02YB37K        2010-07-01      2011-06-30    Male  41.319241   
...          ...               ...             ...     ...        ...   
1197  ID_ZTTHC5X        2010-12-05      2011-12-04    Male  67.000000   
1198  ID_ZUJAFUP        2010-01-14      2011-01-13    Male  43.000000   
1199  ID_ZWHCTUM        2010-07-26      2011-07-25    Male  30.000000   
1200  ID_ZWQRL8L        2010-02-16      2011-02-15    Male  44.000000   
1201  ID_ZWZ92GU        2010-03-18      2011-03-17    Male  28.000000   

     First Transaction Date  No_Pol Car_Category Subject_Car_Colour  \
0                2010-10-23       1  

### Encoding

In [9]:
train.columns

Index(['ID', 'Policy Start Date', 'Policy End Date', 'Gender', 'Age',
       'First Transaction Date', 'No_Pol', 'Car_Category',
       'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State',
       'ProductName', 'target'],
      dtype='object')

In [11]:
categorical_cols = ["Gender", "Car_Category", "Subject_Car_Colour", "Subject_Car_Make", "ProductName", "Policy Start Date", "Policy End Date", "First Transaction Date", "LGA_Name", "State"]

le = LabelEncoder()
for col in categorical_cols:
    test[col] = le.fit_transform(test[col].astype(str))

### Change Dates

In [12]:
train["Policy Start Date"] = pd.to_datetime(train["Policy Start Date"])
train["Policy Start Year"] = train["Policy Start Date"].dt.year
train["Policy Start Month"] = train["Policy Start Date"].dt.month
train["Policy Start Day"] = train["Policy Start Date"].dt.day

In [13]:
train["Policy End Date"] = pd.to_datetime(train["Policy End Date"])
train["Policy End Year"] = train["Policy End Date"].dt.year
train["Policy End Month"] = train["Policy End Date"].dt.month
train["Policy End Day"] = train["Policy End Date"].dt.day

In [14]:
train["First Transaction Date"] = pd.to_datetime(train["First Transaction Date"])
train["First Transaction Year"] = train["First Transaction Date"].dt.year
train["First Transaction Month"] = train["First Transaction Date"].dt.month
train["First Transaction Day"] = train["First Transaction Date"].dt.day

In [ ]:
train = train.drop(["Policy Start Date", "Policy End Date", "First Transaction Date", "ID"], axis=1)

### Putting the target column at the end

In [15]:
target = "target"
train = train[[col for col in train.columns if col != target] + [target]]

In [ ]:
train.head()

In [ ]:
# Check if there any missing values in train set
ax = train.isna().sum().sort_values().plot(kind = 'barh', figsize = (10, 7))
plt.title('Percentage of Missing Values Per Column in Train Set', fontdict={'size':15})
for p in ax.patches:
    percentage ='{:,.0f}%'.format((p.get_width()/train.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))

## Encoding

In [ ]:
categorical_cols = ['Gender', 'Car_Category', 'Subject_Car_Colour', 'Subject_Car_Make', 'LGA_Name', 'State', 'ProductName']
for col in categorical_cols:
        train[col] = LabelEncoder().fit_transform(train[col])

In [ ]:
# Check if there missing values in test set
ax = test.isna().sum().sort_values().plot(kind = 'barh', figsize = (10, 7))
plt.title('Percentage of Missing Values Per Column in Test Set', fontdict={'size':15})

for p in ax.patches:
    percentage ='{:,.1f}%'.format((p.get_width()/test.shape[0])*100)
    width, height =p.get_width(),p.get_height()
    x=p.get_x()+width+0.02
    y=p.get_y()+height/2
    ax.annotate(percentage,(x,y))